<img width="8%" alt="OWID.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/OWID.png" style="border-radius: 15%">

# OWID - Visualize oil consumption throughout the years
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=OWID+-+Visualize+oil+consumption+throughout+the+years:+Error+short+description">Bug report</a>

**Tags:** #dash #dashboard #plotly #naas #asset #analytics #dropdown #callback #bootstrap #snippet

**Author:** [Zihui Ouyang](https://www.linkedin.com/in/zihui-ouyang-539626227/)

**Last update:** 2023-06-26 (Created: 2023-06-26)

**Description:** This notebook creates an interactive plot using Dash app infrastructure with OWID's oil consumption data.

**References:**
- https://ourworldindata.org/grapher/oil-consumption-by-country?tab=chart&time=earliest..latest
- https://stackoverflow.com/questions/70886359/dash-python-making-subplots-when-multiple-parameters-are-selected
- https://dash-example-index.herokuapp.com/line-charts

## Input

### Import libraries

In [ ]:
try:
    import dash
    import os
except:
    !pip install dash --user
    import dash
try:
    import dash_bootstrap_components as dbc
except:
    !pip install dash_bootstrap_components --user
    import dash_bootstrap_components as dbc
import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px

### Setup Variables
- `DASH_PORT`: specify a port number for Dash
- `url`: URL to get data from Excel
- `title`: App title

In [ ]:
DASH_PORT = 8050
url = "https://www.bp.com/content/dam/bp/business-sites/en/global/corporate/xlsx/energy-economics/statistical-review/bp-stats-review-2022-all-data.xlsx"
title = "Oil consumption"

## Model

### Initialize Dash app
The `os.environ.get("JUPYTERHUB_USER")` is used to access the environment variable `JUPYTERHUB_USER` already stored into your Naas Lab.

In [ ]:
app = dash.Dash(
    title=title,
    requests_pathname_prefix=f'/user/{os.environ.get("JUPYTERHUB_USER")}/proxy/{DASH_PORT}/',
    external_stylesheets=[dbc.themes.BOOTSTRAP],
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1.0"}
    ],
)

# app = dash.Dash(title = "Oil consumption") if you are not in Naas

### Get Data

In [ ]:
contents_df = pd.read_excel(url, sheet_name = "Oil Consumption - EJ", header = 2)
contents_df = contents_df.drop([0,5,18,54,64,86,106,111,112,113,114,115,116,117,118,119]) # Drop unnecessary rows
contents_df = contents_df.reset_index(drop=True)
contents_df["Exajoules"] = contents_df["Exajoules"].replace(["of which: OECD"], "OECD")
entity = []  # Lists to create a new dataframe
year = []
oil_consumption = []
for i in range(len(contents_df)):
    for j in range(57):
        entity.append(contents_df.iloc[i,0])
        year.append(1965+j)
        oil_consumption.append(contents_df.iloc[i,j+1])
new_dict = {"Entity": entity,
           "Year": year,
           "oil_consumption": oil_consumption}
new_df = pd.DataFrame(data=new_dict)

### Create Dash App

In [ ]:
p = "oil_consumption"
app.layout = html.Div(
    [
        html.H3("Oil consumption"),
        html.P("Oil consumption is measured in terawatt-hour (TWh) equivalents per year."),
        dcc.Dropdown(
            id="countries",
            options=new_df.Entity.unique(),
            value=["US", "China", "Canada", "United Kingdom", "Australia", "South Africa", "Norway"],
            multi = True
        ),
        dcc.Graph(id=p, figure={}, style={'display': 'none'}),
        dcc.RangeSlider(id='slider', min=1965, max=2021, value=[1965, 2021],
               marks={x: str(x) for x in [1965, 1975, 1985, 1995, 2005, 2021]}),
        html.P('Source: Statistical Review of World Energy - BP (2022)')
    ],
    style={'padding': '40px'}
)

@callback(
    Output(p, 'figure'),
    Output(p, 'style'),
    Input('countries', 'value'),
    Input('slider', 'value')
)

def update_graph(country, year):
    dff = (new_df["Entity"].isin(country)) & (new_df["Year"] <= year[1]) & (new_df["Year"] >= year[0])
    figures = px.line(new_df[dff], x='Year', y=p, color = "Entity", markers = True).update_layout(
                                plot_bgcolor='rgba(0, 0, 0, 0)', height= 600
                            )
    styles = {'display': 'block'}   
    return figures, styles

## Output

### Generate URL and show logs

In [ ]:
if __name__ == "__main__":
    app.run_server(proxy=f"http://127.0.0.1:{DASH_PORT}::https://app.naas.ai")